<a href="https://colab.research.google.com/github/mazabdul7/nlp_coursework_project/blob/main/classifier_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fine tunning model: https://huggingface.co/docs/transformers/training

testing model: https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

In [ ]:
# for colab only
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project



Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project


In [ ]:
pip install datasets

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 47.1 MB/s 
     |████████████████████████████████| 134 kB 49.3 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 127 kB 49.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
     |████████████████████████████████| 144 kB 50.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 34.6 MB/s 
     |████████████████████████████████| 895 kB 27.0 MB/s 
     |████████████████████████████████| 596 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [20]:
from utils.loader import DataLoader
import datasets
import random


In [ ]:
# Load datasets
loader = DataLoader()
train_data = loader.load_amazon(deceptive=False, all=True, test_mode=False)
test_data = loader.load_amazon(deceptive=False, all=True, test_mode=True)

In [21]:
# choose 3 categories to train with
unique_category = train_data['PRODUCT_CATEGORY'].unique()
random.seed(22)
print(unique_category[random.sample(range(30), 3)])
# gogo = train_data[train_data['PRODUCT_CATEGORY'].isin(['Jewelry' 'Outdoors'])]

['Beauty' 'Office Products' 'Grocery']


In [35]:
def df_2_dataset(panda_data, VERIFIED_PURCHASE, PRODUCT_CATEGORY=False):
    if PRODUCT_CATEGORY != False:
        panda_data = panda_data[panda_data['PRODUCT_CATEGORY'].isin(PRODUCT_CATEGORY)]

    panda_data = panda_data[panda_data['VERIFIED_PURCHASE'].isin(VERIFIED_PURCHASE)]

    dataset_data = datasets.Dataset.from_pandas(panda_data)
    remove_columns = ['DOC_ID',
                    'RATING',
                    'VERIFIED_PURCHASE',
                    'PRODUCT_CATEGORY',
                    'PRODUCT_ID',
                    'PRODUCT_TITLE',
                    'REVIEW_TITLE',
                    '__index_level_0__']
    for col in remove_columns:
        dataset_data = dataset_data.remove_columns(col) 
    dataset_data = dataset_data.rename_column('LABEL', 'labels')
    dataset_data = dataset_data.rename_column('REVIEW_TEXT', 'text')

    return dataset_data

verified = ['Y','N']
category = ['Beauty']
df_train = df_2_dataset(train_data, verified, category)
df_test = df_2_dataset(test_data, verified, category)
df_full_test = df_2_dataset(test_data, verified, False)



In [36]:
print(df_train, df_test, df_full_test)

Dataset({
    features: ['labels', 'text'],
    num_rows: 543
}) Dataset({
    features: ['labels', 'text'],
    num_rows: 157
}) Dataset({
    features: ['labels', 'text'],
    num_rows: 5250
})


In [37]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# train and val sets
tokenized_train = df_train.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.train_test_split(test_size=0.1)

# test set
tokenized_test = df_test.map(tokenize_function, batched=True)
tokenized_full_test = df_full_test.map(tokenize_function, batched=True)


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [28]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  # evaluation_strategy="steps",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=5,
                                  save_total_limit = 1,
                                  load_best_model_at_end=True,
                                  save_strategy = "epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
# reduce data size for debugging
# with 100 training data we get ~60%
testing = False

if testing:
  train_dataset = tokenized_train['train'].shuffle(seed=42).select(range(200))
  val_dataset = tokenized_train['test'].shuffle(seed=42).select(range(100))
  test_dataset = tokenized_test.shuffle(seed=42).select(range(200))
  test_full_dataset = tokenized_full_test.shuffle(seed=42).select(range(200))
else:
  train_dataset = tokenized_train['train']
  val_dataset = tokenized_train['test']
  test_dataset = tokenized_test
  test_full_dataset = tokenized_full_test

print(train_dataset, val_dataset, test_dataset,tokenized_full_test)

Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 488
}) Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 55
}) Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 157
}) Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5250
})


In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 488
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 305


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.646499,0.636364
2,No log,0.686214,0.654545
3,No log,0.975456,0.690909
4,No log,1.410461,0.709091
5,No log,1.519723,0.709091


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 55
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-61
Configuration saved in test_trainer/checkpoint-61/config.json
Model weights saved in test_trainer/checkpoint-61/pytorch_model.bin
Deleting older checkpoint [test_trainer/checkpoint-5316] due to args.save_total_limit
Deleting older checkpoint [test_trainer/checkpoint-8860] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 55
  Batch

TrainOutput(global_step=305, training_loss=0.30509130759317366, metrics={'train_runtime': 513.2196, 'train_samples_per_second': 4.754, 'train_steps_per_second': 0.594, 'total_flos': 641990975078400.0, 'train_loss': 0.30509130759317366, 'epoch': 5.0})

In [39]:
from datasets import load_metric
metric = load_metric("glue", "mrpc")

In [42]:
for d in [test_dataset, test_full_dataset]:
    predictions = trainer.predict(d)
    print(predictions.predictions.shape, predictions.label_ids.shape)

    preds = np.argmax(predictions.predictions, axis=-1)
    print(metric.compute(predictions=preds, references=predictions.label_ids))

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 157
  Batch size = 8


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5250
  Batch size = 8


(157, 2) (157,)
{'accuracy': 0.643312101910828, 'f1': 0.7142857142857143}
(5250, 2) (5250,)
{'accuracy': 0.6121904761904762, 'f1': 0.5821018062397373}


In [19]:
path = '/content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_beauty_all'
trainer.save_model(path)


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_all_data
Configuration saved in /content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_all_data/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_all_data/pytorch_model.bin


In [ ]:
new_model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)


loading configuration file /content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_classifier/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/2022-03 SNLP group project/tuned_classifier",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file /content/drive/MyDrive/Colab Notebooks/2

In [ ]:
test_trainer = Trainer(new_model)

ppp = test_trainer.predict(test_dataset)
print(ppp.predictions.shape, ppp.label_ids.shape)

pd = np.argmax(ppp.predictions, axis=-1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 8


(200, 2) (200,)


In [ ]:
metric = load_metric("glue", "mrpc")
metric.compute(predictions=pd, references=ppp.label_ids)

{'accuracy': 0.555, 'f1': 0.611353711790393}